In [1]:
"""
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'
}

configuration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))
"""

"\nimport ibmos2spark\n# @hidden_cell\ncredentials = {\n    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',\n    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',\n    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',\n    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'\n}\n\nconfiguration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'\ncos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')\n\nfrom pyspark.sql import SparkSession\nspark = SparkSession.builder.getOrCreate()\n\ndf = spark.read  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')  .option('header', 'true')  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))\n"

In [2]:
from functions import *
from datetime import datetime

In [3]:
timeSignature = str(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
images_dir = "images/ML"
models_dir = "models"
logs_dir = models_dir+"/"+ timeSignature +"/logs.md"


file = "data/US_Accidents_Dec19.csv"
filter_state = ''

df, sc,spark = setup_spark(file)
createDirectory(models_dir+"/"+timeSignature +"/")

In [4]:
sc

<SparkContext master=local[*] appName=Spark Project>

In [5]:
printSparkConf(sc, logs_dir)

Config: [('spark.driver.extraClassPath', '/opt/conda/lib/python3.7/site-packages/sparkmonitor/listener.jar'), ('SPARKMONITOR_UI_HOST', '192.168.1.228'), ('spark.executor.id', 'driver'), ('spark.driver.host', 'localhost'), ('spark.app.name', 'Spark Project'), ('spark.executor.cores', '4'), ('executor-memory', '6'), ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'), ('spark.num.executors', '6'), ('spark.extraListeners', 'sparkmonitor.listener.JupyterSparkMonitorListener'), ('spark.rdd.compress', 'True'), ('spark.driver.memory', '8g'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.driver.port', '33565'), ('spark.submit.deployMode', 'client'), ('spark.memory.fraction', '.6'), ('spark.ui.showConsoleProgress', 'true'), ('spark.app.id', 'local-1588593951266')]


# Debug tool

In [6]:
df,_ = df.randomSplit([1.0,0.0], 1) # Speed up the program a bit
df.count()

2974335

# 1. Define variables

In [7]:
colLabel = ["Severity"]

colRem = ['ID', 
          'Source',
          'End_Time',
          'End_Lat',
          'End_Lng',
          'Description',
        ]

df, colCat, colNum = setup_variables(df, sc, colLabel, colRem, logs_dir)  

write2file(logs_dir,"Number of rows", str(df.count()))

Label: ['Street', 'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'] 
Categories: ['Street', 'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
Numerical: ['TMC', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Number', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']


# 2. Preprocessing

In [8]:
from pyspark.ml.feature import VectorAssembler,OneHotEncoderEstimator,QuantileDiscretizer, StringIndexer, Imputer,StandardScaler,MinMaxScaler
from pyspark.ml import Pipeline
from numpy import argmax

## Analyse single state
This could be CA since it stands for a little less than 50% of the total set

In [9]:
if filter_state != '':
    df = df.filter(df.State == filter_state) # Lowers the dataset quite a lot
    write2file(logs_dir,"Specified state",str(filter_state))
else:
    write2file(logs_dir,"No state specified","")

## Modify time
Convert the timestamp into a numeric value and then into a string so that the time of day and month can be categorised.

In [10]:
# Convert to int then cast to string

df = df.withColumn('Start_Hour', hour(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Start_Month', month(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df = df.withColumn('Weather_Hour', hour(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Weather_Month', month(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df.show(1)
write2file(logs_dir,"Dataset after modifying UTC timestamp", str(df.take(1)))

+-----+--------+-------------------+---------+----------+------------+------+------+----+------+----------+-----+-------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+----------+-----------+------------+-------------+
|  TMC|Severity|         Start_Time|Start_Lat| Start_Lng|Distance(mi)|Number|Street|Side|  City|    County|State|Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Sunrise_S

## Quantiles function
Alternative solution to get position 
Remove City, Country, State, Zipcode, Airport_Code
In order to reduce dimensionality feature crossing will be applied upon the Start_Lat and Start_Lng then fused in order to create a new set of data

In [11]:
# TODO - Calculate how many buckets needed (Bins)
discretizer_Lat = QuantileDiscretizer(numBuckets=100, inputCol="Start_Lat", outputCol="Start_Lat_disc")
discretizer_Lng = QuantileDiscretizer(numBuckets=100, inputCol="Start_Lng", outputCol="Start_Lng_disc")

# Into categorical values
indexer_cord = [StringIndexer(inputCol=c + "_disc", outputCol=c+"_IDX") for c in ["Start_Lat","Start_Lng"]]

# One-hot (feature crossing)
encoder_cord = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexer_cord], outputCols=["{0}_vec".format(indexer.getOutputCol()) for indexer in indexer_cord])

pipeline_cord = Pipeline(stages=[discretizer_Lat,discretizer_Lng, *indexer_cord, encoder_cord])
preprocessed_cord = pipeline_cord.fit(df).transform(df)

position = VectorAssembler(inputCols=["Start_Lat_IDX_vec","Start_Lng_IDX_vec"], outputCol="position")

df = position.transform(preprocessed_cord)

In [12]:
df.show(1)
write2file(logs_dir,"Dataset after adding quantiles",str(df.take(1)))

+-----+--------+-------------------+---------+----------+------------+------+------+----+------+----------+-----+-------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+----------+-----------+------------+-------------+--------------+--------------+-------------+-------------+-----------------+-----------------+--------------------+
|  TMC|Severity|         Start_Time|Start_Lat| Start_Lng|Distance(mi)|Number|Street|Side|  City|    County|State|Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| Bump|Crossi

## Cast data 
Cast all the datacolumns into correct format so they will be sorted to numerical or categorical values. Data removed caused to big feature vector, contained too many NaN values or were converted into another form.

In [13]:
df = df.select(
        #col('TMC').cast('string'),  Lot of missing values
        col('Severity').cast('int'),
        col('Start_Hour').cast('string'),
        col('Start_Month').cast('string'),
        col('Weather_Hour').cast('string'),
        col('Weather_Month').cast('string'),
        col('position'), # already preprocessed
        col('Distance(mi)').cast('double'),
        col('Side').cast('string'),   
        col('Temperature(F)').cast('double'),
        col('Wind_Chill(F)').cast('double'), #  Lot of missing values
        col('Humidity(%)').cast('double'),
        col('Pressure(in)').cast('double'),
        col('Visibility(mi)').cast('double'),
        col('Wind_Direction').cast('string'),
        col('Wind_Speed(mph)').cast('double'),  #Lot of missing values
        col('Weather_Condition').cast('string'),
        col('Amenity').cast('string'),
        col('Bump').cast('string'),
        col('Crossing').cast('string'),
        col('Give_Way').cast('string'),
        col('Junction').cast('string'),
        col('No_Exit').cast('string'),
        col('Railway').cast('string'),
        col('Roundabout').cast('string'),
        col('Station').cast('string'),
        col('Stop').cast('string'),
        col('Traffic_Calming').cast('string'),
        col('Traffic_Signal').cast('string'),
        col('Turning_Loop').cast('string'),
        col('Sunrise_Sunset').cast('string'),
        col('Civil_Twilight').cast('string'),
        col('Nautical_Twilight').cast('string'),
        col('Astronomical_Twilight').cast('string')
    ) 

# Since the last check can be unorganised we recreate a new list that contains all data
#df.cache()  # TODO does this work? - Heatbeat crash otherwise
colLabel = ["Severity"]

colCat, colNum = createNewClasses(df, sc, colLabel, logs_dir)

Label: ['Start_Hour', 'Start_Month', 'Weather_Hour', 'Weather_Month', 'Side', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'] 
Categories: ['Start_Hour', 'Start_Month', 'Weather_Hour', 'Weather_Month', 'Side', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
Numerical: ['Distance(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']


## Clean data

### Recheck the missing values
Check so that the output contains 0 missing values

In [14]:
printMissingValues(df, logs_dir)

Missing values[Row(Severity=0, Start_Hour=3163, Start_Month=3163, Weather_Hour=36705, Weather_Month=36705, position=0, Distance(mi)=0, Side=0, Temperature(F)=56063, Wind_Chill(F)=1852623, Humidity(%)=59173, Pressure(in)=48142, Visibility(mi)=65691, Wind_Direction=45101, Wind_Speed(mph)=440840, Weather_Condition=65932, Amenity=0, Bump=0, Crossing=0, Give_Way=0, Junction=0, No_Exit=0, Railway=0, Roundabout=0, Station=0, Stop=0, Traffic_Calming=0, Traffic_Signal=0, Turning_Loop=0, Sunrise_Sunset=93, Civil_Twilight=93, Nautical_Twilight=93, Astronomical_Twilight=93)]


### Drop NaN
Cant drop NaN since imputer should take care of the most of them! However we can drop values from the categorical set if we want!

In [15]:
#old = df.count()
#df = df.na.drop()
#print("Rows removed:",old-df.count())

### Recheck the missing values
Check so that the output contains 0 missing values

In [16]:
printMissingValues(df, logs_dir)

Missing values[Row(Severity=0, Start_Hour=3163, Start_Month=3163, Weather_Hour=36705, Weather_Month=36705, position=0, Distance(mi)=0, Side=0, Temperature(F)=56063, Wind_Chill(F)=1852623, Humidity(%)=59173, Pressure(in)=48142, Visibility(mi)=65691, Wind_Direction=45101, Wind_Speed(mph)=440840, Weather_Condition=65932, Amenity=0, Bump=0, Crossing=0, Give_Way=0, Junction=0, No_Exit=0, Railway=0, Roundabout=0, Station=0, Stop=0, Traffic_Calming=0, Traffic_Signal=0, Turning_Loop=0, Sunrise_Sunset=93, Civil_Twilight=93, Nautical_Twilight=93, Astronomical_Twilight=93)]


### Remove data with occurance less than 1%
Based on information from analysis. With further analysis lower procentage can be used to find better results.

In [17]:
original_rows = df.count()
n = int(df.count()*0.001) # Limit the plot to ignore conditions below an limit

weather_freq = df.groupBy('Weather_Condition').count().orderBy('count',ascending=False)
df_filtered = weather_freq.filter(weather_freq['count'] > n)
filtered_conditions = df_filtered.select("Weather_Condition").rdd.flatMap(lambda x: x).collect()
df = df.filter(df['Weather_Condition'].isin(*filtered_conditions))

df_filtered.show()
print("Rows removed:",original_rows - df.count())

write2file(logs_dir,"Weather condition", str(df_filtered.take(df_filtered.count())) + "\nRows removed: " + str(original_rows - df.count()))

+--------------------+------+
|   Weather_Condition| count|
+--------------------+------+
|               Clear|808171|
|       Mostly Cloudy|412528|
|            Overcast|382480|
|                Fair|335289|
|       Partly Cloudy|295439|
|    Scattered Clouds|204662|
|          Light Rain|141073|
|              Cloudy|115496|
|                null| 65932|
|          Light Snow| 42123|
|                Haze| 34315|
|                Rain| 32826|
|                 Fog| 22138|
|          Heavy Rain| 12064|
|       Light Drizzle| 10277|
|Light Thunderstor...|  4928|
|                Snow|  4796|
|        Thunderstorm|  4438|
|        Fair / Windy|  3759|
|               Smoke|  3602|
+--------------------+------+

Rows removed: 103931


## Prepare Pipeline

In [18]:
imputer = Imputer(inputCols=colNum, outputCols=colNum)
imputer.setStrategy("median")

num_assembler = VectorAssembler(inputCols=colNum, outputCol="num_features",handleInvalid="keep")
scaler = MinMaxScaler(min=0.0, max=1.0, inputCol="num_features", outputCol="scaledFeatures")

indexers = [StringIndexer(inputCol = c, outputCol = c +'_IDX', handleInvalid='keep') for c in colCat]

encoder = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexers], outputCols=["{0}_vec".format(indexer.getOutputCol()) for indexer in indexers],handleInvalid="keep")

Imputer_8bc20e492e11

## Preprocessing - Pipeline

In [ ]:
imputer = Imputer(inputCols=colNum, outputCols=colNum)
imputer.setStrategy("median")


num_assembler = VectorAssembler(inputCols=colNum, outputCol="num_features",handleInvalid="keep")
#scaler = StandardScaler(inputCol="num_features", withMean=True, withStd=True, outputCol="scaledFeatures")
scaler = MinMaxScaler(min=0.0, max=1.0, inputCol="num_features", outputCol="scaledFeatures")

indexers = [StringIndexer(inputCol = c, outputCol = c +'_IDX', handleInvalid='keep') for c in colCat]

encoder = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexers], outputCols=["{0}_vec".format(indexer.getOutputCol()) for indexer in indexers],handleInvalid="keep")

In [22]:
# Categorical values

numPipeline = Pipeline(stages=[imputer, num_assembler, scaler])
catPipeline = Pipeline(stages=[*indexers, encoder])

pipeline = Pipeline(stages=[numPipeline, catPipeline])
preprocessed_df = pipeline.fit(df).transform(df)

In [23]:
preprocessed_df.select("position","scaledFeatures",*[c + "_IDX_vec" for c in colCat]).show(1,False)

+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+------------------+-------------------+--------------------+---------------------+-------------+----------------------+-------------------------+---------------+-------------+----------------+----------------+----------------+---------------+---------------+------------------+---------------+-------------+-----------------------+----------------------+--------------------+----------------------+----------------------+-------------------------+-----------------------------+
|position                |scaledFeatures                                                                                                                              |Start_Hour_IDX_vec|Start_Month_IDX_vec|Weather_Hour_IDX_vec|Weather_Month_IDX_vec|Side_IDX_vec |Wind_Direction_IDX_vec|Weather_Condition_IDX_vec|Amenity_IDX_vec|Bump_IDX_vec |Crossing_IDX_

## Combine features

In [24]:
va2 = VectorAssembler(inputCols=["position","scaledFeatures",*[c + "_IDX_vec" for c in colCat]], outputCol="final_features")

df = va2.transform(preprocessed_df)

df = df.withColumn('label', col("Severity"))
df_features = df.withColumn('features', df.final_features).select("features","label")
df_features.show(1, False)

write2fileModel(df_features, models_dir,"df_features", timeSignature)

#df_features.write.format('parquet').mode('overwrite').option("header", "true").save(models_dir+'/df_features.parquet')
write2file(logs_dir,"Feature set size", str(df_features.count()) + "\n\n__Feature vector and label:__\n" + str(df_features.take(1)))
write2file(logs_dir,"Number of rows", str(df_features.count()))
print("Feature set size: ",df_features.count())

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                                                                                       |label|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Count categorical values

In [25]:
# Checking how many classes that can be used
tmp = [df.select(countDistinct(c).alias(c)).collect()[0] for c in [*colCat, "position"]] 
print("Unique column values:", tmp)

write2file(logs_dir,"Unique column values", str(tmp))

Unique column values: [Row(Start_Hour=24), Row(Start_Month=12), Row(Weather_Hour=24), Row(Weather_Month=12), Row(Side=3), Row(Wind_Direction=24), Row(Weather_Condition=19), Row(Amenity=2), Row(Bump=2), Row(Crossing=2), Row(Give_Way=2), Row(Junction=2), Row(No_Exit=2), Row(Railway=2), Row(Roundabout=2), Row(Station=2), Row(Stop=2), Row(Traffic_Calming=2), Row(Traffic_Signal=2), Row(Turning_Loop=1), Row(Sunrise_Sunset=2), Row(Civil_Twilight=2), Row(Nautical_Twilight=2), Row(Astronomical_Twilight=2), Row(position=5406)]


# 3. Feature importance

In [26]:
from pyspark.ml.feature import ChiSqSelector,PCA

## PCA
In order to understand how much the variance affect the dataset we check with PCA. Try to get above 90%

In [27]:
k=250
pca = PCA(k=k, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df_features)
pca_df = pca_model.transform(df_features)
#chi_model[0].pca_features

print("PCA - Feature Variance:","Top 50:\n" + str(pca_model.explainedVariance[:50]) + "\nNumber of items: "+str(k)+"\nSum of variance: "+ str(sum(pca_model.explainedVariance)))
write2file(logs_dir,"PCA - Feature variance", "Top 50:\n" + str(pca_model.explainedVariance[:50]) + "\nNumber of items: "+ str(k) +"\nSum of variance: "+ str(sum(pca_model.explainedVariance)))

PCA - Feature Variance: Top 50:
[0.10947651 0.03672697 0.02766299 0.02552226 0.02237386 0.02085386
 0.02071722 0.02015545 0.01865006 0.01784276 0.01547023 0.01539626
 0.01495166 0.0145945  0.01445063 0.01438479 0.01427268 0.01404841
 0.01390493 0.01257145 0.01244279 0.01156492 0.01133687 0.01110413
 0.01093771 0.01052406 0.00996134 0.00990643 0.00974911 0.00924468
 0.00893532 0.00855816 0.0082812  0.00817473 0.00782579 0.00716821
 0.00624939 0.0061582  0.00578394 0.00564452 0.00531562 0.00511877
 0.0050283  0.00490412 0.00488036 0.00486251 0.00485231 0.00452256
 0.00442257 0.00436215]
Number of items: 250
Sum of variance: 0.9761698682856236


## ChiSqSelector
Check top 100 which of the values in the feature vector

In [28]:
selector = ChiSqSelector(numTopFeatures=k, 
                         labelCol='label', 
                         featuresCol='features', 
                         outputCol="selectedFeatures",
                         selectorType='numTopFeatures', 
                         percentile=0.1, 
                         fpr=0.05, fdr=0.05, fwe=0.05)

chi_model = selector.fit(df_features)
chi_df = chi_model.transform(df_features)

write2file(logs_dir,"Top selected features according to ChiSqSelector", str(chi_model.selectedFeatures)+ "\nNumber of features: " + str(k) + "\nExample data:\n"+str(chi_df.take(5)))
write2fileModel(chi_df, models_dir, "chi_df", timeSignature)

print("Top selected features according to ChiSqSelector:", chi_model.selectedFeatures)
print("Transformed selected features:",chi_df.head().selectedFeatures)
print("Transformed feature vector:", chi_df.show(10))

Saving into: models/2020-05-04_12-05-48/chi_df
Saving through format 1
Top selected features according to ChiSqSelector: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 39, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 

# 4. Machine learning

In [29]:
from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier,RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [30]:
# Use the chi_df for training since it is reduced
#trainSet, testSet = df_features.randomSplit([0.8 ,0.2], 1)
trainSet, testSet = chi_df.randomSplit([0.8 ,0.2], 1)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [31]:
trainSet.show(2,False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                                                      |label|selectedFea

## Logistic regression

In [32]:
lr = LogisticRegression(labelCol="label", featuresCol="selectedFeatures")

paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.1,0.01]) \
                .addGrid(lr.maxIter, [10]) \
                .addGrid(lr.elasticNetParam, [0.6]) \
                .build()
try: 
    _, _ = evaluateModel(lr, paramGrid, "LR_Model", trainSet, testSet, timeSignature, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logs_dir=logs_dir,models_dir=models_dir)
except Exception as e:
    write2file(logs_dir,"Error:", str(e))
    print(e)

Fitting model..
Predicting on testSet..
Evaluating predictions..
Saving into: models/2020-05-04_12-05-48/LR_Model
Saving through format 2
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       2.0|    3|(380,[2,99,198,19...|
|       2.0|    3|(380,[2,99,198,19...|
|       2.0|    2|(380,[2,99,198,19...|
|       2.0|    3|(380,[2,99,198,19...|
|       2.0|    2|(380,[2,99,198,19...|
+----------+-----+--------------------+
only showing top 5 rows

Accuracy:  0.5715905001896042 
Parameters
 {Param(parent='LogisticRegression_4a6d07f3b7e4', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_4a6d07f3b7e4', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_4a6d07f3b7e4', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0

## Decision tree

In [35]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="selectedFeatures") 

paramGrid = ParamGridBuilder().build()

_, _ = evaluateModel(dt, paramGrid, "DT_Model", trainSet, testSet, timeSignature, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logs_dir=logs_dir,models_dir=models_dir)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:33423)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:33423)

## Random forest

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="selectedFeatures") #numTrees=10
paramGrid = ParamGridBuilder() \
                .addGrid(rf.numTrees,[10,15,20,25]) \
                .build()

_, _ = evaluateModel(rf, paramGrid, "RF_Model", trainSet, testSet, timeSignature, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logs_dir=logs_dir,models_dir=models_dir)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:33423)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pyspark/serializ

In [ ]:
write2file("logs_dir","Program finished!", "")

## Hyperparameter tuning of best model

Since there is no predefined way to test multiple algorithms this is used!

In [ ]:
#models = createParams(models)
#bestModels = evaluateAlgorithm(models,trainSet)
#finalModel = getBestModel(bestModels,testSet)

#prediction = makePredictions(finalModel, testSet)
trainingSummary = finalModel.summary

# Make predictions on test documents. cvModel uses the best model found (lrModel).


#trainingSummary = cvModel.bestModel.stages[-1].summary
#selected = prediction.select("label", "probability", "prediction")
#selected.show(10)

In [ ]:
#finalModel = getBestModel(bestModels,testSet)
#print(finalModel)

In [ ]:
accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
falsePositiveRateLabel = trainingSummary.falsePositiveRateByLabel
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
iterations = trainingSummary.totalIterations

print("Iterations: %s\nAccuracy: %s\nFPR: %s\nTPR: %s\nFPRL: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (iterations,accuracy, falsePositiveRate, truePositiveRate, falsePositiveRateLabel, fMeasure, precision, recall))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = prediction.select(['label']).collect()
y_pred = prediction.select(['prediction']).collect()

print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)